In [ ]:
import os
import sys
import pandas as pd
sys.path.append(os.path.dirname(os.getcwd())) #Add project dir to path
from utils.syncUtilities import plotFramesShiftToSyncrhonizeAllSubjectsOneActivity,getMainJointFromMotAndMainBonesFromCSV, getSamplesToSynchronize, SynchronizeAndCutSignals

In [ ]:
import utils.fileProcessing as fileutil 

## Modifies VIDIMU text dataset files for estimated ideal synchronization according to `infoToSync.csv`.

Set dataset root path

In [ ]:
fulldatasetpath = 'D:/VIDIMU'

In [ ]:
inpath = os.path.join(fulldatasetpath,'dataset','videoandimus')
inpathcsv = os.path.join(fulldatasetpath,'analysis','videoandimusync')
outpath = os.path.join(fulldatasetpath,'dataset','videoandimusync')
if not os.path.exists(outpath):
    os.makedirs(outpath)

Load changes neeeded

In [ ]:
csvlog='infoToSync.csv'
csvlogfile = os.path.join(inpathcsv,csvlog)
try:
    dfsync = pd.read_csv(csvlogfile)
except FileNotFoundError:
    print("Can't find ",csvlogfile)

dfsync.head()

In [ ]:
dfsync=dfsync.sort_values(by=['Subject','Activity'])

### Modify text files (.raw, .mot, .csv) --> folder videoandimussync

In [ ]:
subjects = ["S40","S41","S42",  "S44",  "S46","S47","S48","S49",
            "S50","S51","S52","S53","S54","S55","S56","S57"]

In [ ]:
for index,row in dfsync.iterrows():
    subject = row['Subject']
    if subject not in subjects:
        print('Skip subject')
        continue
    file = row['File']
    type = row['Type']
    frames = row['CutFrames']
    outfile=file.replace(inpath,outpath)
    if type == 'mp4':
        #print("Skip video")
        continue
    if not os.path.exists(os.path.dirname(outfile)):
        print("Creating directory: ", os.path.dirname(outfile))
        os.mkdir(os.path.dirname(outfile))
    if type == 'raw':
        lines_per_frame = 5*(50/30) #5 sensors per frame and 50 Hz in IMUs vs 30 Hz in video
        row_start = 2
        fileutil.remove_insidelines_file(file,row_start,int(frames*lines_per_frame),outfile)
    elif type == 'mot':
        lines_per_frame = 50/30 #5 sensors per frame and 50 Hz in IMUs vs 30 Hz in video
        row_start = 9
        fileutil.remove_insidelines_file(file,row_start,int(frames*lines_per_frame),outfile)
    elif type == 'csv':
        lines_per_frame = 1 #video
        row_start = 2
        fileutil.remove_insidelines_file(file,row_start,frames*lines_per_frame,outfile)
    else:
        continue

